# How-to
This notebook serves as a practical guide to common questions users might have.

**Table of content**

1. [Execute moldrug from the command line](#Execute-moldrug-from-the-command-line).
2. [Create your own cost function](#Create-your-own-cost-function).

In [1]:
# Import section
from moldrug.data import ligands, boxes, receptors
import tempfile, os, requests,inspect, gzip, shutil, yaml, sys
from moldrug import utils

## Execute moldrug from the command line

MolDrug it is meant to be for using as a python module. However it has capabilities to work from the command line. Due to the complexity of the input for a normal simulation, [yaml](https://yaml.org/) structured file are used for the input specification. Let first see the help of moldrug.

In [2]:
! moldrug -h

usage: moldrug [-h] [-v] [-f FITNESS] [-o OUTDIR] yaml_file

For information of MolDrug:
    Docs: https://moldrug.readthedocs.io/en/latest/
    Source Code: https://github.com/ale94mleon/moldrug

positional arguments:
  yaml_file             The configuration yaml file

optional arguments:
  -h, --help            show this help message and exit
  -v, --version         show program's version number and exit
  -f FITNESS, --fitness FITNESS
                        The path to the user-custom fitness module; inside of which the given custom cost function must be implemented. See the docs for how to do it properly. E.g. my/awesome/fitness_module.py.By default will look in the moldrug.fitness module.
  -o OUTDIR, --outdir OUTDIR
                        The path to where all the files should be written. By default the current working directory will be used (where the command line was invoked).


As it shows only one positional arguments is needed `yaml_file`. Then you could give:

-   `fitness`: A customized fitness python module where your cost function must be implemented

-   `outdir`: The out directory.

Lets go steep by steep


### yaml_file anatomy

In [3]:
# Getting the data
tmp_path = tempfile.TemporaryDirectory()
lig = ligands.r_x0161
box = boxes.r_x0161
with open(os.path.join(tmp_path.name, 'x0161.pdbqt'), 'w') as f:
    f.write(receptors.r_x0161)


# Getting the CReM data base

url = "http://www.qsar4u.com/files/cremdb/replacements02_sc2.db.gz"
r = requests.get(url, allow_redirects=True)
crem_dbgz_path = os.path.join(tmp_path.name,'crem.db.gz')
crem_db_path = os.path.join(tmp_path.name,'crem.db')
open(crem_dbgz_path, 'wb').write(r.content)
with gzip.open(crem_dbgz_path, 'rb') as f_in:
    with open(crem_db_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print(lig)
print(box)
print(tmp_path.name, os.listdir(tmp_path.name))

COC(=O)C=1C=CC(=CC1)S(=O)(=O)N
{'A': {'boxcenter': [12.11, 1.84, 23.56], 'boxsize': [22.5, 22.5, 22.5]}}
/tmp/tmp2_shy776 ['crem.db.gz', 'x0161.pdbqt', 'crem.db']


Now we have all that we need. The SMILES, the definition of the box and the receptor. But first let see what are the arguments that hte function `utils.Local` needs:

In [4]:
inspect.getfullargspec(utils.Local.__init__).args

['self',
 'mol',
 'crem_db_path',
 'costfunc',
 'grow_crem_kwargs',
 'costfunc_kwargs']

We will pass all of this parameters using config.yaml file.

**Warning!**: Check your correct temporal file and change it accordantly. Of course you could also set a normal directory and safe the results of the simulation.


Here is an example yaml file to work with the class `utils.Local`.

```yaml
main:
  type: Local
  njobs: 1
  pick: 1
  mol: COC(=O)C=1C=CC(=CC1)S(=O)(=O)N
  costfunc: Cost
  costfunc_kwargs:
    vina_executable: vina
    receptor_path: /tmp/your_tmp_dir/x0161.pdbqt
    boxcenter:
      - 12.11
      - 1.84
      - 23.56
    boxsize:
      - 22.5
      - 22.5
      - 22.5
    exhaustiveness: 4
    ncores: 12
    num_modes: 1
  crem_db_path: /tmp/your_tmp_dir/crem.db
  grow_crem_kwargs:
    radius: 3
    min_atoms: 1
    max_atoms: 2
    ncores: 12
```
The structure of a yaml file is like a python dictionary but more human readable (see this [youtube video](https://www.youtube.com/watch?v=1uFVr15xDGg&list=PL6ebkIZFT4xXiVdpOeKR4o_sKLSY0aQf_&index=11) if you are not familiar).

First we have the directrix:

```yaml
main:
```

This is just the name of your main project (we will se that `moldrug.utils.GA` accept also follow projects) and could be any word. Then we have:

```yaml
  type: Local
  njobs: 1
  pick: 1
```

Looks how this section is inside of `main` (one indentation level). `type` is the name of the inside `moldrug.utils`; for now could be GA or Local (case insensitive). In this case we want to use the class `Local`. `njobs` and `pick` are the parameters when the class `Local` (or `GA`) is call. The rest of the parameters are just the parameters that needs `Local` for the initialization. All of them are at the same level of the previous ones. Because `costfunc_kwargs` is a dictionary; we add for its value a new indentation level:

```yaml
  costfunc_kwargs:
    vina_executable: vina
    receptor_path: /tmp/your_tmp_dir/x0161.pdbqt
    boxcenter:
      - 12.11
      - 1.84
      - 23.56
    boxsize:
      - 22.5
      - 22.5
      - 22.5
    exhaustiveness: 4
    ncores: 12
    num_modes: 1
```
As you could imagine tha keyword boxcenter is a list and this is one of the way to represent list in yaml. Cool right?

The next dictionary is the same that the yaml file and we will save it in the temporal file that we created.

In [5]:
config = {
    "main": {
        "type": "Local",
        "njobs": 1,
        "pick": 1,
        "mol": lig,
        "costfunc": "Cost",
        "costfunc_kwargs": {
            "vina_executable": "vina",
            "receptor_path": os.path.join(tmp_path.name, 'x0161.pdbqt'),
            "boxcenter": [
                12.11,
                1.84,
                23.56
            ],
            "boxsize": [
                22.5,
                22.5,
                22.5
            ],
            "exhaustiveness": 4,
            "ncores": 12,
            "num_modes": 1
        },
        "crem_db_path": os.path.join(tmp_path.name, 'crem.db'),
        "grow_crem_kwargs": {
            "radius": 3,
            "min_atoms": 1,
            "max_atoms": 2,
            "ncores": 12
        }
    }
}

# Save the config as a yaml file
with open(os.path.join(tmp_path.name, 'config.yml'), 'w') as f:
    yaml.dump(config, f)
os.listdir(tmp_path.name)

['crem.db.gz', 'x0161.pdbqt', 'config.yml', 'crem.db']

### Move to the directory and run moldrug

In [6]:
cwd = os.getcwd()
os.chdir(tmp_path.name)
! moldrug config.yml
os.chdir(cwd)
os.listdir(tmp_path.name)
os.chdir(cwd)

The main job is being executed.
Calculating cost function...
100%|█████████████████████████████████████████████| 2/2 [00:06<00:00,  3.08s/it]
File local_pop.sdf was createad!
The main job finished!.


As you see the simulation run successfully and the following files were generated:

1.  `local_result.pbz2`; the compresses version of the `Local` class with all the information of the simulation (binary file).

2.  `local_pop.sdf`; the mol structures. This file is useful to use inside Pymol.

### Specify a custom cost function

The following is a simple cost function based on the vina score and very similar to `moldrug.fitness.Cost` used by **MolDrug**. The details on the parameters and how to correctly implement it will be discussed letter on the [following section](#create-your-own-cost-function).


```python
def MyAwesomeCost(Individual:utils.Individual, wd:str = '.vina_jobs', vina_executable:str = 'vina', receptor_path:str = None, boxcenter:List[float] = None, boxsize:List[float] =None, exhaustiveness:int = 8, ncores:int = 1,  num_modes:int = 1):

    # Getting Vina score
    cmd = f"{vina_executable} --receptor {receptor_path} --ligand {os.path.join(wd, f'{Individual.idx}.pdbqt')} "\
        f"--center_x {boxcenter[0]} --center_y {boxcenter[1]} --center_z {boxcenter[2]} "\
        f"--size_x {boxsize[0]} --size_y {boxsize[1]} --size_z {boxsize[2]} "\
        f"--out {os.path.join(wd, f'{Individual.idx}_out.pdbqt')} --cpu {ncores} --exhaustiveness {exhaustiveness} --num_modes {num_modes}"

    # Creating the ligand pdbqt
    with open(os.path.join(wd, f'{Individual.idx}.pdbqt'), 'w') as l:
        l.write(Individual.pdbqt)
    utils.run(cmd)

    # Getting the information
    best_energy = utils.VINA_OUT(os.path.join(wd, f'{Individual.idx}_out.pdbqt')).BestEnergy()
    # Changing the 3D conformation by the conformation of the binding pose
    Individual.pdbqt = ''.join(best_energy.chunk)

    # Getting the Scoring function of Vina
    Individual.vina_score = best_energy.freeEnergy

    # Getting the Scoring function of Vina and assign it to the cost attribute of the Individual
    Individual.cost = best_energy.freeEnergy
    return Individual
```

Because this cost function accept the same parameters we can use the same that Cost; but the name of the function is different. So we have to modify that parameter. Let's save the function in a .py file with the corresponded import statements 


In [7]:
# Create a new config file
# Save the config as a yaml file
config['main']['costfunc'] = 'MyAwesomeCost'
with open(os.path.join(tmp_path.name, 'config_custom_fitness.yml'), 'w') as f:
    yaml.dump(config, f)
os.listdir(tmp_path.name)

['local_result.pbz2',
 'crem.db.gz',
 'x0161.pdbqt',
 'config.yml',
 'local_pop.sdf',
 'crem.db',
 'config_custom_fitness.yml']

In [8]:
str_module = """#!/usr/bin/env python3
# -*- coding: utf-8 -*-
from moldrug import utils
from typing import List
import os

def MyAwesomeCost(Individual:utils.Individual, wd:str = '.vina_jobs', vina_executable:str = 'vina', receptor_path:str = None, boxcenter:List[float] = None, boxsize:List[float] =None, exhaustiveness:int = 8, ncores:int = 1,  num_modes:int = 1):

    # Getting Vina score
    cmd = f"{vina_executable} --receptor {receptor_path} --ligand {os.path.join(wd, f'{Individual.idx}.pdbqt')} "\
        f"--center_x {boxcenter[0]} --center_y {boxcenter[1]} --center_z {boxcenter[2]} "\
        f"--size_x {boxsize[0]} --size_y {boxsize[1]} --size_z {boxsize[2]} "\
        f"--out {os.path.join(wd, f'{Individual.idx}_out.pdbqt')} --cpu {ncores} --exhaustiveness {exhaustiveness} --num_modes {num_modes}"

    # Creating the ligand pdbqt
    with open(os.path.join(wd, f'{Individual.idx}.pdbqt'), 'w') as l:
        l.write(Individual.pdbqt)
    utils.run(cmd)

    # Getting the information
    best_energy = utils.VINA_OUT(os.path.join(wd, f'{Individual.idx}_out.pdbqt')).BestEnergy()
    # Changing the 3D conformation by the conformation of the binding pose
    Individual.pdbqt = ''.join(best_energy.chunk)

    # Getting the Scoring function of Vina
    Individual.vina_score = best_energy.freeEnergy

    # Getting the Scoring function of Vina and assign it to the cost attribute of the Individual
    Individual.cost = best_energy.freeEnergy
    return Individual
"""
# Saving the new fitness
with open(os.path.join(tmp_path.name, 'MyCustomFitness.py'), 'w') as f:
    f.write(str_module)
os.listdir(tmp_path.name)

['local_result.pbz2',
 'crem.db.gz',
 'x0161.pdbqt',
 'config.yml',
 'local_pop.sdf',
 'crem.db',
 'MyCustomFitness.py',
 'config_custom_fitness.yml']

Now we only need to call MolDrug and specify the new fitness module to use. It is recommendable to output in on folder the results; therefore also use the option `outdir` of the command line.

In [9]:
cwd = os.getcwd()
os.chdir(tmp_path.name)
! moldrug config_custom_fitness.yml --fitness MyCustomFitness.py --outdir custom_fitness
os.listdir(tmp_path.name)
os.chdir(cwd)

The main job is being executed.
Calculating cost function...
100%|█████████████████████████████████████████████| 2/2 [00:05<00:00,  2.95s/it]
File local_pop.sdf was createad!
The main job finished!.


In [10]:
print(os.listdir(tmp_path.name))
print(os.listdir(os.path.join(tmp_path.name, 'custom_fitness')))

['local_result.pbz2', 'crem.db.gz', 'custom_fitness', 'x0161.pdbqt', 'config.yml', 'local_pop.sdf', 'crem.db', 'MyCustomFitness.py', 'config_custom_fitness.yml']
['local_result.pbz2', 'CustomMolDrugFitness.py', 'local_pop.sdf', '__pycache__']


As you can see the simulation runs successfully! Here is the possible issue. Because the cost function used is not actually inside of MolDrug. If we would like to use on the future the binary file `local_result.pbz2`. We have first say to Python where to find the used finesse function. For reproducibility and also peace of mind. MolDrug will generate a new file: `CustomMolDrugFitness.py` this is nothing more than a copy of your implemented python fitness module. And with this module is the one used for the internal calculation of MolDrug. Therefore, we have to say to python where is `CustomMolDrugFitness` and append to sys.path in order to open the binary file. If we do not du that we get an error. Let see the example.

In [11]:


print(f"We are in the directory: {os.getcwd()}. Here are the following files: {os.listdir()}")
# Adding the path where CustomMolDrugFitness.py is.

try:
    whole_result = utils.decompress_pickle(os.path.join(tmp_path.name, 'custom_fitness', 'local_result.pbz2'))
except Exception as e:
    print(f"This was the problem: {e}. So we have to add to the system path the directory where CustomMolDrugFitness.py is located.")   
    sys.path.append(os.path.join(tmp_path.name, 'custom_fitness'))
    whole_result = utils.decompress_pickle(os.path.join(tmp_path.name, 'custom_fitness', 'local_result.pbz2'))

whole_result


We are in the directory: /home/ale/GITLAB/moldrug/docs/notebooks. Here are the following files: ['how_to.ipynb', '.ipynb_checkpoints', 'quickstart.ipynb']
This was the problem: No module named 'CustomMolDrugFitness'. So we have to add to the system path the directory where CustomMolDrugFitness.py is located.


In [12]:
# The cost modify cost function
whole_result.costfunc

<function CustomMolDrugFitness.MyAwesomeCost(Individual: moldrug.utils.Individual, wd: str = '.vina_jobs', vina_executable: str = 'vina', receptor_path: str = None, boxcenter: List[float] = None, boxsize: List[float] = None, exhaustiveness: int = 8, ncores: int = 1, num_modes: int = 1)>

In [13]:
# The original cost function form the first simulation
(utils.decompress_pickle(os.path.join(tmp_path.name, 'local_result.pbz2'))).costfunc

<function moldrug.fitness.Cost(Individual: moldrug.utils.Individual, wd: str = '.vina_jobs', vina_executable: str = 'vina', receptor_path: str = None, boxcenter: List[float] = None, boxsize: List[float] = None, exhaustiveness: int = 8, ncores: int = 1, num_modes: int = 1)>

You could see the difference between the two cost functions and the error printed in the try-except block. Finally let's take a look to the sys.path; check that at the end it is our path with our new fitness module.

In [14]:
sys.path

['/home/ale/GITLAB/moldrug/docs/notebooks',
 '/home/ale/anaconda3/envs/moldrug/lib/python39.zip',
 '/home/ale/anaconda3/envs/moldrug/lib/python3.9',
 '/home/ale/anaconda3/envs/moldrug/lib/python3.9/lib-dynload',
 '',
 '/home/ale/anaconda3/envs/moldrug/lib/python3.9/site-packages',
 '/home/ale/GITLAB/moldrug',
 '/tmp/tmp2_shy776/custom_fitness']

## Create your own cost function

Create your own cost function should be straightforward as soon as we understand how MolDrug works with it. 